# Clase 14:

## Parte 1

En esta clase aprenderemos: 
- Usar python
- Integrar Python con ArcGIS
- Usar Chat GPT para ambas cosas



In [4]:
import platform

# Print the Python version
print("Python version:", platform.python_version())


Python version: 3.9.11


In [5]:
import os

# Obtener el directorio de trabajo actual
current_directory = os.getcwd()

print("Directorio actual:", current_directory)


Directorio actual: C:\Users\af.camacho1169\Documents\ArcGIS\Projects\MyProject


In [16]:
# Defino directorio principal
import arcpy 
import os

gdb_path=r"C:\Users\af.camacho1169\Documents\ArcGIS\Projects\MyProject\start\Tornado.gdb"
# Set the workspace to your geodatabase
arcpy.env.workspace = gdb_path

#Lista de las objetos dentro de mi path
feature_classes = arcpy.ListFeatureClasses()

# Loop through each feature class and add it to the current map
project = arcpy.mp.ArcGISProject("CURRENT")
map = project.listMaps()[0]  # Assuming you want to add to the first map in the project

for fc in feature_classes:
    layer = map.addDataFromPath(os.path.join(gdb_path, fc))
    print(f"Added {fc} to the map.")



Added parcels to the map.
Added schools to the map.
Added streets to the map.
Added tornado to the map.
Added Eltornado_Buffer to the map.
Added ElTornadoBuffer_200_Meters to the map.
Added ElTornadoBuffer_300_Meters to the map.
Added ElTornadoBuffer_100_Meters to the map.


In [12]:
def remove_non_basemap_layers():
    # Get the current project and the first map
    project = arcpy.mp.ArcGISProject("CURRENT")
    map = project.listMaps()[0]

    # Iterate through all layers
    for layer in map.listLayers():
        # Check if the layer is not a basemap
        if not layer.isBasemapLayer:
            # Remove the layer
            map.removeLayer(layer)
            #print(f"Removed layer: {layer.name}")

    # Save the project
    project.save()

    print("All non-basemap layers have been removed.")

# Call the function
#remove_non_basemap_layers()


In [15]:
remove_non_basemap_layers()

All non-basemap layers have been removed.


## Queremos estimar un área de influencia usando
- Buffer (area de influencia) al rededor de la capa tornado:
    - 500 metros al rededor de la zona de influencia
    - end point: redondo
- Seleccionar todas las parcelas que cruzan con el área de influencia

In [16]:
# Correctly format the output path
output_feature_class = os.path.join(gdb_path, "Eltornado_Buffer")
distancia_del_buffer="250 Meters"

In [17]:
arcpy.analysis.Buffer("tornado",
                      output_feature_class,
                      distancia_del_buffer,
                      "FULL",
                      "ROUND",
                      "ALL",
                      None,
                      "GEODESIC")

<Result 'C:\\Users\\af.camacho1169\\Documents\\ArcGIS\\Projects\\MyProject\\start\\Tornado.gdb\\Eltornado_Buffer'>

### La iteración de los loops

In [2]:
# Vector de distancais
distancias = ["100 Meters", "200 Meters", "300 Meters"]

In [11]:
import arcpy
#Loop que recrorra esas distintas distancias 
for d in distancias: 
    #Formatear la ruta de salida 
    out_file=os.path.join(gdb_path, f"ElTornadoBuffer_{d.replace(' ','_')}")

    # Llamo la función buffer 
    arcpy.analysis.Buffer("tornado",
                      out_file,
                      d,
                      "FULL",
                      "ROUND",
                      "ALL",
                      None,
                      "GEODESIC")

## Selección de los predios dañados por el tornado

Queremos estimar el 80% del valor de las parcelas que están dentro de la zona de influencia del buffer:

Para esto podemos usar la herramienta spatial join, podemos también usar seleccionar por ubicación. 

In [23]:
import arcpy
import os

gdb_path=r"C:\Users\af.camacho1169\Documents\ArcGIS\Projects\MyProject\start\Tornado.gdb"
# Set the workspace to your geodatabase
arcpy.env.workspace = gdb_path
arcpy.env.overwriteOutput=True

distancias = ["100 Meters", "200 Meters", "300 Meters"]

#Deseleccionar cualquier capa: 
arcpy.management.SelectLayerByAttribute('parcels', 'CLEAR_SELECTION')


#Loop que recrorra esas distintas distancias 
for d in distancias: 
    #Formatear la ruta de salida 
    out_file=os.path.join(gdb_path, f"FTornadoBuffer_{d.replace(' ','_')}")
    
    if arcpy.Exists(out_file):
                          arcpy.Delete_management(out_file)
                          
    # Llamo la función buffer 
    arcpy.analysis.Buffer("tornado",
                      out_file,
                      d,
                      "FULL",
                      "ROUND",
                      "ALL",
                      None,
                      "GEODESIC")
                          
                          
    #Seleccionamos parcelas que se intersectan con ese buffer: 
                          
    arcpy.management.SelectLayerByLocation('parcels', "INTERSECT", out_file)
                          
    #Sumamos el valor de los edificios 
    suma_valor_edificio=0
                          
    with arcpy.da.SearchCursor('parcels', ["BLDGVAL"]) as cursor: 
                          for row in cursor:
                              suma_valor_edificio+=row[0]
    #El 80% del valor total: 
    valor_80_pc=suma_valor_edificio*0.8
                          
    #Mostramos el resultado
    print(f"Dstancia {d} , valor dañado : {valor_80_pc}")
                          

Dstancia 100 Meters , valor dañado : 19600440.0
Dstancia 200 Meters , valor dañado : 24898287.200000003
Dstancia 300 Meters , valor dañado : 33231825.6
